<a href="https://colab.research.google.com/github/peppievt/Gated-training/blob/main/Pre_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from matplotlib import style

import librosa
import librosa.display
import IPython.display as ipd
import keras

from numpy import dot
from numpy.linalg import norm
from google.colab import drive

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import LSTM, Embedding
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization

from keras.utils import np_utils, to_categorical
import pickle
from keras.layers import TimeDistributed

from sklearn.preprocessing import MinMaxScaler
import scipy.sparse as sparse
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity

import tensorflow as tf

from tqdm import tnrange, tqdm_notebook, tqdm

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#adres = r'C:\Users\pepij\Documents\Uni\Datascience\Thesis\baldey_audio\real_compound'
#in colab:
adres = r'/content/gdrive/My Drive/Colab Notebooks/data/baldey_audio/'
opslag_adres = r'/content/gdrive/My Drive/Colab Notebooks/data/'
learning_Rate = 0.002
#style.use('dark_background')
#options = ['real_derived', 'real_underived', 'real_compound', 'pseudo_derived', 'pseudo_underived', 'pseudo_compound']
options = ['BALDEY_variants/real_derived']
#options2 = ['BALDEY_variants/real_derived', 'BALDEY_variants/real_underived', 'BALDEY_variants/real_compound', 'BALDEY_variants/pseudo_derived', 'BALDEY_variants/pseudo_underived', 'BALDEY_variants/pseudo_compound']

In [ ]:
def print_sound_wave(x):
    for i in dir_list[0:x]:
        fname = dir_list + '/' + i  
        data, sampling_rate = librosa.load(fname)
        plt.figure(figsize=(15, 5))
        librosa.display.waveplot(data, sr=sampling_rate)
        
def create_Y():
    woordenlijst = []
    for i in dir_list:
        if i.endswith('.wav'):
            woordenlijst.append(i[:-4])
    return woordenlijst

def create_RandomVector():
    Vec = np.zeros(30)
    randomlist = random.sample(range(0, 30), ones)
    for i in randomlist:
        Vec[i] = 1
    return np.asarray(Vec)

def create_Vectors():
    Y = []
    y = create_Y()
    y2 = []
    for i in range(len(y)):
        x = create_RandomVector()
        Y.append(x)
    return Y

def create_map_Vectors(woordenlijst):
    omzet_df = pd.DataFrame(columns = ['woord', 'vector'])
    woorden, vectoren = [],[]
    for i in set(woordenlijst):
        woorden.append(i)
        vectoren.append(create_RandomVector())
    omzet_df['woord'] = woorden
    omzet_df['vector'] = vectoren
    dictionary = dict(zip(omzet_df['woord'],omzet_df['vector']))
    return dictionary

def solve_table(df, di):
    df['Y'] = df['name'].map(di)
    return df

In [ ]:
df_features = pd.DataFrame(columns=['feature'])
counter =0

def generate_features(options):
    df_features = pd.DataFrame(columns=['feature'])
    names = []
    counter =0
    c=0
    for opt in range(len(options)):
        dir_list = os.listdir(adres+options[opt])
        print("\noptie = ", options[opt])
        print("lengte = ", len(dir_list))
        for i in dir_list[0:10]:
            if counter % 100 == 0:
                print("counter: ", counter, i)
            X, sample_rate = librosa.load((adres+options[opt]) + '/'+ i
                                    #duration #check max of fragment. Niet A-priori doen, laten bepalen door het algoritme. achteraf bijstellen.
                                    ,sr=None #native is more convenient.
                                    ,offset=0
                                    )
            sample_rate = np.array(sample_rate)
            samp = int(sample_rate/100)
            specto = librosa.feature.melspectrogram(y=X, sr=sample_rate, hop_length=441)
            #take the logarithm -> to decibel (MFSC):
            specto = librosa.amplitude_to_db(specto, ref=np.max)
            df_features.loc[counter] = [specto]
            names.append(i.split('.')[0])
            counter=counter+1 
    return df_features, names

def get_names(options):
    names = []
    counter =0
    for opt in range(len(options)):
        dir_list = os.listdir(adres+options[opt])
        print("optie = ", options[opt])
        for i in dir_list:
            names.append(i.split('.')[0])
            counter=counter+1 
    return names

In [ ]:
opties = options
df_features, names = generate_features(opties)


optie =  BALDEY_variants/real_derived
lengte =  2280
counter:  0 beteugelde.wav_110


In [ ]:
print(df_features['feature'][0].shape)

(128, 77)


In [ ]:
df_features.to_pickle(opslag_adres + 'fake_2_specto_baldey.pkl') 
df_name = pd.DataFrame(names)
df_name.to_pickle(opslag_adres +'fake_2_specto_baldey_names.pkl')
print("check dataset even lang als losse sets: ", len(df_features))
print("lengte names", len(df_name))